In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mbkinaci/image-localization-dataset

Saving kaggle.json to kaggle.json
  0% 0.00/5.40M [00:00<?, ?B/s]
100% 5.40M/5.40M [00:00<00:00, 49.8MB/s]


In [2]:
!pip install xmltodict

In [3]:
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patheffects as PathEffects
import random

from PIL import Image
from xml.dom import minidom
from xml.etree import cElementTree as ET
from lxml import etree
import xmltodict

from lxml import etree

In [4]:
with zipfile.ZipFile('image-localization-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [5]:
clases = ['background', 'pepino', 'berenjena', 'champiñón']

In [6]:
datos = []

PATH = '/content/training_images/training_images'

for i in os.listdir(PATH):
    datos.append(i)
len(datos)

372

In [7]:
datos.sort()
imagenes = []
etiquetas = []
for i in range(len(datos)):
    if i % 2 == 0:
        imagenes.append(Image.open(PATH+'/'+i))
    else:
        etiquetas.append(datos[i])
len(imagenes), len(etiquetas)

TypeError: ignored

In [ ]:
from collections import defaultdict
from xml.etree import cElementTree as ET

In [ ]:
def xml2dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

#convert xml to str to dict

In [ ]:
etiquetas_dict = []

for i in etiquetas:
    #from .xml to str
    tree = etree.parse(PATH+'/'+i)
    xml_str = etree.tostring(tree, encoding='utf8', method='xml')

    xml_data = ET.XML(xml_str)

    #from str to dict
    #dic = xmltodict.parse(xml_str)
    d = xml2dict(xml_data)
    #agregando a lista
    etiquetas_dict.append(dic)

In [ ]:
type(etiquetas_dict[0])

In [ ]:
etiquetas_dict[0]

In [ ]:
d = {}
for i in etiquetas_dict[0]['annotation']:
    print(i)

In [ ]:
etiquetas_dict[0]['annotation']

In [ ]:
def get_sample(ix):
  img, label = imagenes, etiquetas_dict
  img_np = np.array(img)
  anns = label['annotation']['object']
  if type(anns) is not list:
    anns = [anns]
  labels = np.array([classes.index(ann['name']) for ann in anns])
  bbs = [ann['bndbox'] for ann in anns]
  bbs = np.array([[int(bb['xmin']), int(bb['ymin']),int(bb['xmax'])-int(bb['xmin']),int(bb['ymax'])-int(bb['ymin'])] for bb in bbs])
  anns = (labels, bbs)
  return img_np, anns

def plot_anns(img, anns, ax=None, bg=-1):
  # anns is a tuple with (labels, bbs)
  # bbs is an array of bounding boxes in format [x_min, y_min, width, height] 
  # labels is an array containing the label 
  if not ax:
    fig, ax = plt.subplots(figsize=(10, 6))
  ax.imshow(img)
  labels, bbs = anns
  for lab, bb in zip(labels, bbs):
    if bg == -1 or lab != bg:
      x, y, w, h = bb
      rect = mpatches.Rectangle((x, y), w, h, fill=False, edgecolor='red', linewidth=2)
      text = ax.text(x, y - 10, classes[lab], {'color': 'red'})
      text.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])
      ax.add_patch(rect)

In [ ]:
img_np, anns = get_sample(5)
plot_anns(img_np, anns)
plt.show()